In [1]:
from pynq import PL
from pynq import Overlay
from time import sleep

overlay = Overlay("bram_lzw.bit")
overlay.download()
print(overlay.ip_dict)

from pynq import MMIO

# Memory map everything
lzw_addr    = 0x40000000
lzw         = MMIO(lzw_addr, 4194304)

def lzw_start():
    lzw.write(0x0, 1) # ap_start
    
def lzw_status():
    print("LZW Status:", lzw.read(0x00)) # status of overlay
    
def lzw_set_in_size(in_size):
    lzw.write(0x80000, in_size)
    
def lzw_print_status():
    print("-----------------------------------")
    in_size      = lzw.read(0x80000)
    out_size     = lzw.read(0x80008)
    print("IN_SIZE:  ", in_size)
    print("OUT_SIZE: ", out_size)
    print("\n-----------------------------------")        
    
def bytes_to_int(x):
    y   = []
    num = 0
    # ex: 03020100
    for i, byte in enumerate(x):
        # little endian I think
        if   i%4 == 0:
            num = 0
            num |= byte
        elif i%4 == 1:
            num |= byte << 8
        elif i%4 == 2:
            num |= byte << 16
        else:
            num |= byte << 24
            y.append(int(num))
            
    if (len(x)%4 != 0):
        # there's leftover from the above loop
        y.append(num)
    return y

{'SEG_bram_decompress_0_Reg': [1073741824, 4194304, None]}


In [2]:
compressed = []
data_size  = 0

# read file
file = open("weights/5-0-e.bin", "rb")
compressed = file.read()

# packing 4 chars/bytes into 1 int
compressed = bytes_to_int(compressed)
# IF THE ENCODED FILE STARTS WITH "0C00 0000" WE NEED TO POP THEM
# ELSE WE DO NOT NEED TO POP
compressed.pop(0)
total_size = len(compressed)

print("Printing input bytes, packed into 32 bits:")
for i in range(10):
    print(i, hex(compressed[i]))

print("Total Size: ", total_size)

lzw_set_in_size(total_size*4)

# write to buffer
for i in range(total_size):
    lzw.write(0x20000 + (i*4), compressed[i])

lzw_start()
sleep(3)

0 0x4129c818
1 0x4101298
2 0x300a9356
3 0xd2814a78
4 0x40421bd6
5 0xe0795389
6 0x3e1814b7
7 0xa0086f16
8 0x451b0475
9 0x35c30ac5
Total Size:  29350


In [7]:
lzw_print_status()

# size of out_size in 32 bit integers
out_ints = int(lzw.read(0x80008)/4)

import numpy as np

byte4_mask = 0xFF000000
byte3_mask = 0x00FF0000
byte2_mask = 0x0000FF00
byte1_mask = 0x000000FF

for i in range(out_ints):
    reg   = lzw.read(0x40000 + (i*4))
    byte1 = (reg & byte1_mask)
    byte2 = (reg & byte2_mask) >> 8 
    byte3 = (reg & byte3_mask) >> 16
    byte4 = (reg & byte4_mask) >> 24

    if i>0 and i%4==0:
        print()
    print("{:02X}{:02X} {:02X}{:02X} ".format(byte1, byte2, byte3, byte4), end='')
print()

-----------------------------------
IN_SIZE:   117400
OUT_SIZE:  147456

-----------------------------------
5F3D 902F 0000 0000 A948 4D2F 0000 0000 
9CE5 D232 0000 0000 916B 7457 0000 0000 
2CF4 AFEC 0000 0000 197E 1787 0000 0000 
522F B576 0000 0000 7ECC 91C2 0000 0000 
F3B5 5C17 0000 0000 2986 C16F 0000 0000 
3E55 6326 0000 0000 80E8 6467 0000 0000 
4074 614E 0000 0000 967D 8F9C 0000 0000 
1626 0521 0000 0000 B455 2E63 0000 0000 
9437 A0F3 0000 0000 8DC6 C16A 0000 0000 
1572 DCD6 0000 0000 195B 65EF 0000 0000 
E0B6 455E 0000 0000 9E6D B7BF 0000 0000 
1BA2 A5C0 0000 0000 B57C 98E5 0000 0000 
FF99 27B5 0000 0000 A146 4961 0000 0000 
7801 8544 0000 0000 A74B 5911 0000 0000 
7854 1E08 0000 0000 880E 7795 0000 0000 
AC35 9573 0000 0000 B0F9 BB07 0000 0000 
29E7 2F19 0000 0000 2D30 E861 0000 0000 
4CC0 23D0 0000 0000 6EE5 55D4 0000 0000 
1A75 2664 0000 0000 26E1 FD80 0000 0000 
5F31 5761 0000 0000 92F5 D249 0000 0000 
D052 151A 0000 0000 2D63 5F69 0000 0000 
1D81 F0F6 0000 0000 D869 7C40 